# Data preprocessing

In [29]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import os

In [30]:
from gensim.models import word2vec

In [31]:
current_directory = os.getcwd()
print(current_directory)

c:\Users\seowoo kim\Desktop\노나없


In [32]:
# Japan first_read japan data
data_j = pd.read_csv("./intern_review_china_only_sentences.csv")
data_j.head()

,SeparatedSentences
0,Why even such a beautiful and successful woman...
1,It all depends on De Niro.The first half looke...
2,"For me, the world will be a better place if I ..."
3,When Hathaway intervened in Streep's strongwom...
4,It's been a long time since I've seen Princess...


# Preprocessing Strategy

1. Expand contractions : pycontractions
2. lowercase
3. change numbers to words : inflect
4. Remove Punctuations : '!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'
5. Remove remained emoji, japanese, chinese, korean words
6. PoS_tagging: from nltk import pos_tag
7. If needed, remove PRP, TO, DT, etc. 
8. Stemming: PorterStemmer
9. Find out average sentence length
10. Wordcloud


In [33]:
# Pycontractions doesn't work due to dependency issue. I just modulized it by myself.

from contractions import expand_contractions

contracted_list = []
Sentences = data_j['SeparatedSentences'].copy()

for sentence in Sentences:
    expanded_sentence = expand_contractions(sentence)
    contracted_list.append(expanded_sentence)


In [34]:
contracted_list[:5]

['Why even such a beautiful and successful woman in American movies has to accept such an ordinary and cheating husband?',
 "It all depends on De Niro.The first half looked good, the second half values swallowed conservative and retreated.De Niro's people are a little too perfect.",
 'For me, the world will be a better place if I can make one or two such warm skits every year that look good and release tears.。。。。',
 "When Hathaway intervened in Streep's strongwoman life, now De Niro anti-invasion Hathaway life……actually a little ambitious, trying to discuss career women and the older generation, but floating a bit.I like Hathaway a lot, but every little expression of her has a sense of rehearsal and release, especially compared to De Niro's relaxed share.And De Niro is really handsome...…………",
 'it is been a long time since i have seen Princess Anne, but the movie feels numb, especially after "Fashion Witch" has set the bar so high for professional women and young people, it is also to

In [35]:
# Transform all sentences into lower case! easy!

contracted_list = [text.lower() for text in contracted_list]

In [36]:
contracted_list[:5]

['why even such a beautiful and successful woman in american movies has to accept such an ordinary and cheating husband?',
 "it all depends on de niro.the first half looked good, the second half values swallowed conservative and retreated.de niro's people are a little too perfect.",
 'for me, the world will be a better place if i can make one or two such warm skits every year that look good and release tears.。。。。',
 "when hathaway intervened in streep's strongwoman life, now de niro anti-invasion hathaway life……actually a little ambitious, trying to discuss career women and the older generation, but floating a bit.i like hathaway a lot, but every little expression of her has a sense of rehearsal and release, especially compared to de niro's relaxed share.and de niro is really handsome...…………",
 'it is been a long time since i have seen princess anne, but the movie feels numb, especially after "fashion witch" has set the bar so high for professional women and young people, it is also to

In [37]:
# Change numbers into words
import inflect
p = inflect.engine()

def convert_number(text):
    def replace_number(match):
        number = match.group()
        return p.number_to_words(number)
    
    converted_text = re.sub(r'\d+', replace_number, text)
    return converted_text

inflected_list = []
inflected_list = [convert_number(text) for text in contracted_list]         

In [38]:
# Remove punctuations
rm_punctionations = inflected_list.copy()

def rm_punct(text):
    cleaned_text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', ' ', text)
    return cleaned_text

rm_punctionations = [rm_punct(text) for text in rm_punctionations]

In [39]:
rm_punctionations

['why even such a beautiful and successful woman in american movies has to accept such an ordinary and cheating husband ',
 'it all depends on de niro the first half looked good  the second half values swallowed conservative and retreated de niro s people are a little too perfect ',
 'for me  the world will be a better place if i can make one or two such warm skits every year that look good and release tears 。。。。',
 'when hathaway intervened in streep s strongwoman life  now de niro anti invasion hathaway life……actually a little ambitious  trying to discuss career women and the older generation  but floating a bit i like hathaway a lot  but every little expression of her has a sense of rehearsal and release  especially compared to de niro s relaxed share and de niro is really handsome   …………',
 'it is been a long time since i have seen princess anne  but the movie feels numb  especially after  fashion witch  has set the bar so high for professional women and young people  it is also to

In [40]:
# remove japanese, korean, chinese words

def only_english(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    return cleaned_text

rm_punctionations = [only_english(text) for text in rm_punctionations]

In [41]:
# POS_tagging

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# 필요한 NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Text를 단어 토큰과 POS 태그로 변환
def pos_tagging(text):
    word_tokens = word_tokenize(text)
    return pos_tag(word_tokens)

tag_sentence = rm_punctionations.copy()

tag_sentence = [pos_tagging(text) for text in tag_sentence]


[nltk_data] Downloading package punkt to C:\Users\seowoo
[nltk_data]     kim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\seowoo kim\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [42]:
tag_sentence

[[('why', 'WRB'),
  ('even', 'RB'),
  ('such', 'PDT'),
  ('a', 'DT'),
  ('beautiful', 'NN'),
  ('and', 'CC'),
  ('successful', 'JJ'),
  ('woman', 'NN'),
  ('in', 'IN'),
  ('american', 'JJ'),
  ('movies', 'NNS'),
  ('has', 'VBZ'),
  ('to', 'TO'),
  ('accept', 'VB'),
  ('such', 'PDT'),
  ('an', 'DT'),
  ('ordinary', 'JJ'),
  ('and', 'CC'),
  ('cheating', 'JJ'),
  ('husband', 'NN')],
 [('it', 'PRP'),
  ('all', 'DT'),
  ('depends', 'VBZ'),
  ('on', 'IN'),
  ('de', 'IN'),
  ('niro', 'FW'),
  ('the', 'DT'),
  ('first', 'JJ'),
  ('half', 'NN'),
  ('looked', 'VBD'),
  ('good', 'JJ'),
  ('the', 'DT'),
  ('second', 'JJ'),
  ('half', 'NN'),
  ('values', 'NNS'),
  ('swallowed', 'VBN'),
  ('conservative', 'JJ'),
  ('and', 'CC'),
  ('retreated', 'VBD'),
  ('de', 'FW'),
  ('niro', 'FW'),
  ('s', 'JJ'),
  ('people', 'NNS'),
  ('are', 'VBP'),
  ('a', 'DT'),
  ('little', 'JJ'),
  ('too', 'RB'),
  ('perfect', 'JJ')],
 [('for', 'IN'),
  ('me', 'PRP'),
  ('the', 'DT'),
  ('world', 'NN'),
  ('will', 'MD'),


In [43]:
nltk.download('tagsets')
nltk.help.upenn_tagset('WRB')

WRB: Wh-adverb
    how however whence whenever where whereby whereever wherein whereof why


[nltk_data] Downloading package tagsets to C:\Users\seowoo
[nltk_data]     kim\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [44]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Remove IN, PRP, CC, PRP$, DT, TO, FW, WP
def remove_specific_tags(tagged_sentence, tags_to_remove):
    return [word for word, tag in tagged_sentence if tag not in tags_to_remove]

tags_to_remove = {"IN", "PRP", "CC", "PRP$", "DT", "TO", "FW", "WP", "WRB"}

tag_rm = tag_sentence.copy()
tag_rm = [remove_specific_tags(text, tags_to_remove) for text in tag_rm]

In [45]:
tag_rm_sentence = [pos_tag(sentence) for sentence in tag_rm]
tag_rm_sentence[:5]

[[('even', 'RB'),
  ('such', 'JJ'),
  ('beautiful', 'JJ'),
  ('successful', 'JJ'),
  ('woman', 'NN'),
  ('american', 'JJ'),
  ('movies', 'NNS'),
  ('has', 'VBZ'),
  ('accept', 'NNS'),
  ('such', 'JJ'),
  ('ordinary', 'JJ'),
  ('cheating', 'NN'),
  ('husband', 'NN')],
 [('depends', 'NNS'),
  ('first', 'JJ'),
  ('half', 'NN'),
  ('looked', 'VBD'),
  ('good', 'JJ'),
  ('second', 'JJ'),
  ('half', 'NN'),
  ('values', 'NNS'),
  ('swallowed', 'VBD'),
  ('conservative', 'JJ'),
  ('retreated', 'JJ'),
  ('s', 'NN'),
  ('people', 'NNS'),
  ('are', 'VBP'),
  ('little', 'JJ'),
  ('too', 'RB'),
  ('perfect', 'JJ')],
 [('world', 'NN'),
  ('will', 'MD'),
  ('be', 'VB'),
  ('better', 'JJR'),
  ('place', 'NN'),
  ('i', 'NN'),
  ('can', 'MD'),
  ('make', 'VB'),
  ('one', 'CD'),
  ('two', 'CD'),
  ('such', 'JJ'),
  ('warm', 'JJ'),
  ('skits', 'NNS'),
  ('year', 'NN'),
  ('look', 'VBP'),
  ('good', 'JJ'),
  ('release', 'NN'),
  ('tears', 'NNS')],
 [('hathaway', 'NN'),
  ('intervened', 'VBD'),
  ('streep',

In [46]:
# Lemmatize & remove stopwords

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import wordnet as wn

nltk.download('wordnet')

# inflect engine initialize
p = inflect.engine()

# WordNetLemmatizer initialize
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None

def singularize_and_lemmatize(tagged_words):
    new_words = []
    
    for word, tag in tagged_words:
        if tag.startswith('N'):
            singular_word = p.singular_noun(word)
            if singular_word:
                new_words.append(singular_word)
            else:
                new_words.append(word)
        else:
            wn_tag = get_wordnet_pos(tag)
            if wn_tag is not None:
                lemma_word = lemmatizer.lemmatize(word, wn_tag)
                new_words.append(lemma_word)
            else:
                new_words.append(word)
    
    return new_words

[nltk_data] Downloading package wordnet to C:\Users\seowoo
[nltk_data]     kim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
processed_sentences = [singularize_and_lemmatize(sentence) for sentence in tag_rm_sentence]

In [48]:
processed_sentences

[['even',
  'such',
  'beautiful',
  'successful',
  'woman',
  'american',
  'movie',
  'have',
  'accept',
  'such',
  'ordinary',
  'cheating',
  'husband'],
 ['depend',
  'first',
  'half',
  'look',
  'good',
  'second',
  'half',
  'value',
  'swallow',
  'conservative',
  'retreated',
  's',
  'person',
  'be',
  'little',
  'too',
  'perfect'],
 ['world',
  'will',
  'be',
  'good',
  'place',
  'i',
  'can',
  'make',
  'one',
  'two',
  'such',
  'warm',
  'skit',
  'year',
  'look',
  'good',
  'release',
  'tear'],
 ['hathaway',
  'intervene',
  'streep',
  's',
  'strongwoman',
  'life',
  'now',
  'de',
  'anti',
  'invasion',
  'hathaway',
  'life',
  'actually',
  'little',
  'ambitious',
  'try',
  'discuss',
  'career',
  'woman',
  'old',
  'generation',
  'float',
  'bit',
  'i',
  'hathaway',
  'lot',
  'little',
  'expression',
  'have',
  'sense',
  'rehearsal',
  'release',
  'especially',
  'compare',
  'de',
  'share',
  'de',
  'be',
  'really',
  'handsome']

In [49]:
# stopwords
def remove_verbs(words):
    verbs_to_remove = {"am", "be", "are", "was", "were", "being", "is"}
    filtered_words = [word for word in words if word not in verbs_to_remove]
    return filtered_words

processed_sentences_no_stopwords = [remove_verbs(word) for word in processed_sentences]

In [50]:
processed_sentences_no_stopwords

[['even',
  'such',
  'beautiful',
  'successful',
  'woman',
  'american',
  'movie',
  'have',
  'accept',
  'such',
  'ordinary',
  'cheating',
  'husband'],
 ['depend',
  'first',
  'half',
  'look',
  'good',
  'second',
  'half',
  'value',
  'swallow',
  'conservative',
  'retreated',
  's',
  'person',
  'little',
  'too',
  'perfect'],
 ['world',
  'will',
  'good',
  'place',
  'i',
  'can',
  'make',
  'one',
  'two',
  'such',
  'warm',
  'skit',
  'year',
  'look',
  'good',
  'release',
  'tear'],
 ['hathaway',
  'intervene',
  'streep',
  's',
  'strongwoman',
  'life',
  'now',
  'de',
  'anti',
  'invasion',
  'hathaway',
  'life',
  'actually',
  'little',
  'ambitious',
  'try',
  'discuss',
  'career',
  'woman',
  'old',
  'generation',
  'float',
  'bit',
  'i',
  'hathaway',
  'lot',
  'little',
  'expression',
  'have',
  'sense',
  'rehearsal',
  'release',
  'especially',
  'compare',
  'de',
  'share',
  'de',
  'really',
  'handsome'],
 ['long',
  'time',
  

In [51]:
# final..refine!

def replace_words(sentence):
    replacements = {
        r'\binternben\b': ['intern', 'ben'],
        r'\bexperiencesben\b': ['experience', 'ben'],
        r'\bbeingben\b': ['being', 'ben'],
        r'\bbentwo\b': ['ben', 'two'],
        r'\bbens\b': ['ben'],
        r'\bitben\b': ['it', 'ben'],
        r'\bheartben\b': ['heart', 'ben'],
        r'\botherben\b': ['other', 'ben'],
        r'\bbentheir\b': ['ben', 'their'],
        r'\bfamilyben\b': ['family', 'ben'],
        r'\bfunben\b': ['fun', 'ben'],
        r'\binternrobert\b': ['intern', 'robert'],
        r'\buprobert\b': ['up', 'robert'],
        r'\brealisticrobert\b': ['realistic', 'robert'],
        r'\btemperrobert\b': ['temper', 'robert'],
        r'\bplacerobert\b': ['place', 'robert'],
        r'\btimesrobert\b': ['time', 'robert'],
        r'\bniros\b': ['niro'],
        r'\bdeniro\b': ['niro'],
        r'\bniroi\b': ['niro', 'I'],
        r'\bmyershe\b': ['she'],
        r'\bmyersworking\b': ['working'],
        r'\banhathaway\b': ['hathaway'],
        r'\bsceneanhathaway\b': ['scene', 'hathaway'],
        r'\banhathawayhis\b': ['hathaway', 'his'],
        r'\bhathawayde\b': ['hathaway', 'de'],
        r'\bhathaways\b': ['hathaway'],
        r'\banhathaways\b': ['hathaway'],
        r'\brottenanhathaway\b': ['rotten', 'hathaway'],
        r'\btherejules\b': ['there', 'jule'],
        r'\bweakjule\b': ['weak', 'jule'],
        r'\bjules\b': ['jule'],
        r'\broutinejule\b': ['routine', 'jule'],
        r'\bagejule\b': ['age', 'jule'],
    }
    for pattern, replacement in replacements.items():
        sentence = re.sub(pattern, ' '.join(replacement), sentence)

    return sentence.split()

modified_sentences = [replace_words(' '.join(sentence)) for sentence in processed_sentences_no_stopwords]

In [52]:
modified_sentences

[['even',
  'such',
  'beautiful',
  'successful',
  'woman',
  'american',
  'movie',
  'have',
  'accept',
  'such',
  'ordinary',
  'cheating',
  'husband'],
 ['depend',
  'first',
  'half',
  'look',
  'good',
  'second',
  'half',
  'value',
  'swallow',
  'conservative',
  'retreated',
  's',
  'person',
  'little',
  'too',
  'perfect'],
 ['world',
  'will',
  'good',
  'place',
  'i',
  'can',
  'make',
  'one',
  'two',
  'such',
  'warm',
  'skit',
  'year',
  'look',
  'good',
  'release',
  'tear'],
 ['hathaway',
  'intervene',
  'streep',
  's',
  'strongwoman',
  'life',
  'now',
  'de',
  'anti',
  'invasion',
  'hathaway',
  'life',
  'actually',
  'little',
  'ambitious',
  'try',
  'discuss',
  'career',
  'woman',
  'old',
  'generation',
  'float',
  'bit',
  'i',
  'hathaway',
  'lot',
  'little',
  'expression',
  'have',
  'sense',
  'rehearsal',
  'release',
  'especially',
  'compare',
  'de',
  'share',
  'de',
  'really',
  'handsome'],
 ['long',
  'time',
  

In [53]:
# remove empty list
modified_sentences = [sentence for sentence in modified_sentences if sentence]

In [54]:
## remove words with len=1

modified_sentences_2 = modified_sentences.copy()

modified_sentences_2 = [[i for i in word if len(i) > 1] for word in modified_sentences_2]


modified_sentences_2[:10]


[['even',
  'such',
  'beautiful',
  'successful',
  'woman',
  'american',
  'movie',
  'have',
  'accept',
  'such',
  'ordinary',
  'cheating',
  'husband'],
 ['depend',
  'first',
  'half',
  'look',
  'good',
  'second',
  'half',
  'value',
  'swallow',
  'conservative',
  'retreated',
  'person',
  'little',
  'too',
  'perfect'],
 ['world',
  'will',
  'good',
  'place',
  'can',
  'make',
  'one',
  'two',
  'such',
  'warm',
  'skit',
  'year',
  'look',
  'good',
  'release',
  'tear'],
 ['hathaway',
  'intervene',
  'streep',
  'strongwoman',
  'life',
  'now',
  'de',
  'anti',
  'invasion',
  'hathaway',
  'life',
  'actually',
  'little',
  'ambitious',
  'try',
  'discuss',
  'career',
  'woman',
  'old',
  'generation',
  'float',
  'bit',
  'hathaway',
  'lot',
  'little',
  'expression',
  'have',
  'sense',
  'rehearsal',
  'release',
  'especially',
  'compare',
  'de',
  'share',
  'de',
  'really',
  'handsome'],
 ['long',
  'time',
  'have',
  'see',
  'princess

In [55]:
lengths = [len(sentence) for sentence in modified_sentences_2]

# sentence lenths discriptive analysis
median = np.median(lengths)
mean = np.mean(lengths)
percentile_75 = np.percentile(lengths, 75)
percentile_90 = np.percentile(lengths, 90)
percentile_95 = np.percentile(lengths, 95)

print ('median: ', median, 
       'mean: ', mean, 
       'per_75: ', percentile_75, 
       'per_90: ', percentile_90, 
       'per_95: ', percentile_95)

median:  11.0 mean:  15.915710068875041 per_75:  24.0 per_90:  38.0 per_95:  45.0


In [56]:
#wordcloud
from wordcloud import WordCloud
text_data_china = " ".join([" ".join(sentence) for sentence in modified_sentences_2])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_data_china)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [57]:
from collections import Counter
all_words = [word for sentence in modified_sentences_2 for word in sentence]
word_freq= Counter(all_words)
print(word_freq)

Counter({'movie': 1177, 'have': 1165, 'deployment': 938, 'not': 927, 'old': 780, 'do': 612, 'life': 579, 'intern': 531, 'ben': 489, 'year': 461, 'work': 443, 'person': 425, 'may': 394, 'can': 386, 'man': 384, 'spoiler': 351, 'film': 337, 'good': 309, 'see': 303, 'think': 296, 'very': 293, 'hathaway': 289, 'time': 288, 'woman': 287, 'seventy': 272, 'watch': 271, 'love': 269, 'one': 264, 'anne': 248, 'company': 243, 'there': 241, 'so': 234, 'thing': 225, 'young': 219, 'two': 214, 'really': 214, 'go': 211, 'first': 205, 'will': 202, 'say': 201, 'also': 194, 'jule': 191, 'new': 191, 'want': 185, 'still': 177, 'more': 172, 'story': 172, 'warm': 171, 'experience': 171, 'just': 169, 'look': 164, 'always': 161, 'feel': 160, 'make': 155, 'too': 151, 'up': 150, 'age': 143, 'workplace': 136, 'robert': 136, 'family': 134, 'find': 133, 'that': 131, 'play': 130, 'get': 129, 'female': 128, 'saw': 124, 'other': 123, 'most': 121, 'day': 121, 'retirement': 121, 'know': 118, 'only': 116, 'many': 115, 'su

In [58]:
df_word_freq = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
df_word_freq.sort_values(by='Frequency', ascending=False, inplace=True)
df_word_freq

,Word,Frequency
6,movie,1177
7,have,1165
1402,deployment,938
86,not,927
52,old,780
...,...,...
2683,marxism,1
2679,latitude,1
2677,utopian,1
2676,inauthenticity,1


In [59]:
#! pip install openpyxl
df_word_freq.to_excel('word_freq_china.xlsx')

# word2Vec

In [60]:
from gensim.models import Word2Vec

In [61]:
model = Word2Vec(sentences=modified_sentences_2, vector_size=100, window=46, min_count=2, workers=1)

In [62]:
model.save("word2vec_china.model")

In [63]:
word_vectors = model.wv

In [64]:
similar_words = word_vectors.most_similar('intern')
print(similar_words)

[('tell', 0.9996979236602783), ('jule', 0.9996786713600159), ('young', 0.9996684193611145), ('man', 0.9996260404586792), ('retire', 0.9996221661567688), ('year', 0.999612033367157), ('senior', 0.9995865225791931), ('return', 0.9995791912078857), ('company', 0.9995740056037903), ('work', 0.9995317459106445)]


In [65]:
similar_words_100 = word_vectors.most_similar('ben', topn=100)
print("Top 100 similar words:", similar_words_100)


Top 100 similar words: [('company', 0.999725878238678), ('life', 0.999705970287323), ('young', 0.9997026920318604), ('old', 0.999679446220398), ('seventy', 0.9996506571769714), ('work', 0.9994608759880066), ('intern', 0.9994293451309204), ('do', 0.9994200468063354), ('man', 0.9994081258773804), ('love', 0.9993481040000916), ('jule', 0.9993374347686768), ('person', 0.9993230104446411), ('year', 0.9993120431900024), ('retire', 0.9992993474006653), ('can', 0.9992215633392334), ('return', 0.9992035627365112), ('not', 0.9991207122802734), ('forty', 0.9990859031677246), ('deployment', 0.9990657567977905), ('senior', 0.9990550875663757), ('tell', 0.9990204572677612), ('woman', 0.999006450176239), ('shop', 0.9990022778511047), ('founder', 0.9989935755729675), ('husband', 0.9989170432090759), ('commerce', 0.9986854791641235), ('retirement', 0.9986492991447449), ('balance', 0.9985286593437195), ('try', 0.9984424710273743), ('death', 0.9984356760978699), ('live', 0.9983811974525452), ('nearly', 0

# WEAT

In [67]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [68]:
df_word_freq = pd.read_excel('C:/Users/seowoo kim/Desktop/노나없/word_freq_china.xlsx')

In [69]:
df_word_freq_new = df_word_freq[df_word_freq['Frequency']>1]

In [70]:
from transformers import pipeline

In [71]:
model = pipeline('sentiment-analysis', model="finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
c:\Users\seowoo kim\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [72]:
sentiment_label = []
confidence_score = []

# Analyze sentiment and store results
for word in df_word_freq_new['Word']:
    result = model(word)
    sentiment_label.append(result[0]['label'])
    confidence_score.append(result[0]['score'])

# Add the sentiment results to the original DataFrame
df_word_freq_new['Sentiment Label'] = sentiment_label
df_word_freq_new['Confidence Score'] = confidence_score

# Display the updated DataFrame
print(df_word_freq_new)

      Unnamed: 0         Word  Frequency Sentiment Label  Confidence Score
0              6        movie       1177             NEU          0.898441
1              7         have       1165             NEU          0.853081
2           1402   deployment        938             NEU          0.862611
3             86          not        927             NEU          0.831674
4             52          old        780             NEU          0.858976
...          ...          ...        ...             ...               ...
2448        1221       advise          2             NEU          0.880438
2449        3497    therefore          2             NEU          0.953520
2450        3464      rapidly          2             NEU          0.855164
2451         337    enjoyment          2             POS          0.883375
2452        1239  connotation          2             NEU          0.937682

[2453 rows x 5 columns]


C:\Users\seowoo kim\AppData\Local\Temp\ipykernel_2084\3591878570.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_freq_new['Sentiment Label'] = sentiment_label
C:\Users\seowoo kim\AppData\Local\Temp\ipykernel_2084\3591878570.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_freq_new['Confidence Score'] = confidence_score


In [73]:
df_word_freq_new.to_excel('sentiment_word_china.xlsx')

In [79]:
NEU = df_word_freq_new[df_word_freq_new['Sentiment Label']=='NEU']
len(NEU)
POS = df_word_freq_new[df_word_freq_new['Sentiment Label']=='POS']
len(POS)
NEG = df_word_freq_new[df_word_freq_new['Sentiment Label']=='NEG']
len(NEG)

72

In [77]:
from gensim.models import Word2Vec
model_path = "C:/Users/seowoo kim/Desktop/노나없/word2vec_china.model"
model = Word2Vec.load(model_path)

In [90]:
old_words = ['ben', 'senior', 'old', 'elderly', 'elder']
young_words = ['jule', 'junior', 'young']

In [91]:
pos_words = [
    "gentleman", "strong", "handsome", "smart", "intelligent", "succes", "successful",
    "healthy", "kindness", "powerful", "talented", "charm", "courage", "encourage", "respect", "respectful"
    "gentle", "brave", "optimistic", "warmth", "friendly", "attractive", "sympathetic",
    "victorious", "hero", "heroine", "courteous", "wise", "wisdom", "passion", "passionate", "lovely", "beloved"
]
neg_words = [
    "arrogant", "hypocritical", "unscrupulous", "stupid", "childish", 
    "betrayal", "bully", "angry", "fearful", "rude", "untrustworthy", 
    "stubborn", "tacky", "negative", "annoying", "selfish", "shameful", 
    "ridiculous", "pathetic"
]

In [92]:
# cosine similarity 
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))

def s(w, A, B):
    c_a = np.array([cos_sim(w, a) for a in A])
    c_b = np.array([cos_sim(w, b) for b in B])
    mean_A = np.mean(c_a)
    mean_B = np.mean(c_b)
    return mean_A - mean_B

# Target A - Target B
def s_group(X, A, B):
    return np.array([s(x, A, B) for x in X])

#WEAT
def weat_score(X, Y, A, B):
    s_X = s_group(X, A, B)
    s_Y = s_group(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return (mean_X - mean_Y) / std_dev

In [93]:
X = np.array([model.wv.get_vector(word) for word in old_words if word in model.wv])
Y = np.array([model.wv.get_vector(word) for word in young_words if word in model.wv])
A = np.array([model.wv.get_vector(word) for word in pos_words if word in model.wv])
B = np.array([model.wv.get_vector(word) for word in neg_words if word in model.wv])

In [94]:
# WEAT score result
weat_result = round(weat_score(X, Y, A, B), 3)
print("WEAT Score:", weat_result)

WEAT Score: 1.11


In [95]:
def weat_score(X, Y, A, B):
    s_X = s_group(X, A, B)
    s_Y = s_group(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    # 표준 편차 출력
    print("Standard Deviation:", std_dev)
    
    if std_dev == 0:
        return float('nan')
    
    return (mean_X - mean_Y) / std_dev

X = np.array([model.wv.get_vector(word) for word in old_words if word in model.wv])
Y = np.array([model.wv.get_vector(word) for word in young_words if word in model.wv])
A = np.array([model.wv.get_vector(word) for word in pos_words if word in model.wv])
B = np.array([model.wv.get_vector(word) for word in neg_words if word in model.wv])

# WEAT score result
weat_result = round(weat_score(X, Y, A, B), 3)
print("WEAT Score:", weat_result)

Standard Deviation: 0.00090258213
WEAT Score: 1.11
